In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style('ticks')

In [26]:
train = pd.read_excel('semifinal_data/train.xlsx')
test = pd.read_excel('semifinal_data/test.xlsx')
descr = pd.read_excel('semifinal_data/description.xlsx')

In [3]:
from nltk import ngrams
from collections import defaultdict
from gensim.models import Word2Vec

word2vec

In [27]:
train['Full_descr'] = train['Shrt_Desc'].values + ' ' +\
    train['GmWt_Desc1'].fillna('').values + ' ' +\
    train['GmWt_Desc2'].fillna('').values

In [28]:
train = train.drop(['Shrt_Desc', 'GmWt_Desc1', 'GmWt_Desc2'], axis = 1)

In [29]:
words = train['Full_descr']

In [30]:
def foo(word):
    temp = word.lower().split()
    res = []
    for i in temp:
        res += i.split(',')
    return res

In [31]:
sent = [foo(row) for row in words]

In [32]:
size_ = 10
model = Word2Vec(sent, min_count=1, size=size_, workers=5,
                 window=4, sg = 2, iter = 5)

In [33]:
temp = words.apply(lambda x :
                   np.array([model[i] for i in foo(x)]).mean(axis = 0))

/home/data_sapiens/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [34]:
w2v_columns =[]
for i in range(size_):
    train['w2v_{}'.format(i)] = temp.apply(lambda x : x[i])
    w2v_columns += ['w2v_{}'.format(i)]

In [12]:
from sklearn.neighbors import KNeighborsRegressor as KNN

In [13]:
for i in range(5):
    knn_model = KNN(n_neighbors = 10 + i*5)
    knn_model.fit(train[w2v_columns], train['Energ_Kcal'])
    temp = knn_model.predict(train[w2v_columns])
    train['knn_{}'.format(5 + i*5)] = temp

In [35]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression

In [36]:
all_names = train.columns.values

In [37]:
class full_model:
    def __init__(self):
        self.xgb = xgb.XGBRegressor(silent=True)
        self.lin = LinearRegression()
        #self.lin2 = LinearRegression()
        
    def fit(self, X, y):
        self.xgb.fit(X, y, eval_metric = mean_absolute_error)
        self.lin.fit(X, y)
        #self.lin2.fit(np.concatenate((self.xgb.predict(X).reshape(-1, 1),
        #            self.lin.predict(X).reshape(-1, 1)), axis=1), y)
        
    def predict(self, X):
        #return self.lin2.predict(np.concatenate((self.xgb.predict(X).reshape(-1, 1),
        #            self.lin.predict(X).reshape(-1, 1)), axis=1))
        return 0.5 * (self.xgb.predict(X) + self.lin.predict(X))

In [38]:
train = train.fillna(0)

In [39]:
def trainXGB(names_list):
    X_train, X_test, y_train, y_test = train_test_split(
        train[names_list].drop(['Energ_Kcal', 'Full_descr'],
            axis=1), train['Energ_Kcal'], test_size=0.1, random_state=42)
    
    kf = KFold(n_splits=5, random_state=None, shuffle=True)
    ans = []
    ans2 = []
    for train_index, val_index in kf.split(X_train):
        model = full_model()
        X_ktrain, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_ktrain, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        model.fit(X_ktrain.values, y_ktrain.values)
        ans += [mean_absolute_error(model.predict(X_test.values), y_test)]
        ans2 += [mean_absolute_error(model.predict(X_val.values), y_val)]
        
        #
        #
        #break
        
    return np.array(ans), np.array(ans2)

In [43]:
train = pd.read_excel('semifinal_data/train.xlsx')
test = pd.read_excel('semifinal_data/test.xlsx')
descr = pd.read_excel('semifinal_data/description.xlsx')

In [51]:
from scripts.semifinal_features import prepare_features

def trainXGB2(names_list):
    train1, test1, scaler = prepare_features(train, test, False)
    X_train, X_test, y_train, y_test = train_test_split(
        train1.drop('Energ_Kcal', axis=1).values, train1['Energ_Kcal'].values, test_size=0.1, random_state=42)
    
    kf = KFold(n_splits=5, random_state=None, shuffle=True)
    ans = []
    ans2 = []
    for train_index, val_index in kf.split(X_train):
        model = full_model()
        X_ktrain, X_val = X_train[train_index], X_train[val_index]
        y_ktrain, y_val = y_train[train_index], y_train[val_index]
        model.fit(X_ktrain, y_ktrain)
        ans += [mean_absolute_error(model.predict(X_test), y_test)]
        ans2 += [mean_absolute_error(model.predict(X_val), y_val)]
        
        #
        #
        #break
        
    return np.array(ans), np.array(ans2)

In [52]:
all_res, all_res_val = trainXGB2(all_names)
all_res

/home/data_sapiens/programs/challenges/best_hack/scripts/semifinal_features.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cur_col + '_encoded'][test_idx] = train[cur_col][test_idx].map(mapping)
/home/data_sapiens/Anaconda/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


array([6.85769726, 6.73032268, 6.63413454, 6.90109311, 6.76433272])

In [53]:
all_res_val

array([6.50304825, 6.2460121 , 6.00528176, 5.86428019, 6.6409122 ])

In [21]:
def get_ngramsrank(n):
    gram2number = defaultdict(lambda : 0);
    for word in words:
        n_grams = ngrams(word, n)
        for i in n_grams:
            gram2number["".join(i)] += 1

    max_score = 0
    for i in gram2number.items():
        if i[1] > max_score:
            max_score = i[1]

    gram2rank = dict()
    for i in gram2number.items():
        gram2rank[i[0]] = int(i[1])
    
    return gram2rank

In [22]:
example = get_ngramsrank(8)

In [23]:
def prepare_word(word):
    res = 0
    n_grams = ngrams(word, n)
    for i in n_grams:
        res += 1 / koks["".join(i)]
    return res

In [24]:
train['Ranking_Desc'] = train['Full_descr'].apply(prepare_word)

NameError: name 'n' is not defined

In [ ]:
train.corr()['Ranking_Desc']['Energ_Kcal']